In [261]:
!pip install transformers

In [262]:
import pandas as pd
import regex as re
import string
import pandas as pd
import time
import os

In [263]:
!pip install sastrawi

In [327]:
#********************************************************************************************************#
# Data Taken From : https://github.com/rifkyahmadsaputra/Sentiment-Analysis-Online-Lectures-in-Indonesia #
#********************************************************************************************************#

df = pd.read_csv('https://raw.githubusercontent.com/rifkyahmadsaputra/Sentiment-Analysis-Online-Lectures-in-Indonesia/main/data/25k_tweets_data_clean_polarity.csv')
df = df[['text_clean', 'polarity_score', 'polarity']]
df.dropna(inplace=True)
df.head()

,text_clean,polarity_score,polarity
0,iya sih tapi maksud sender kuliah online kali,2,positive
1,males kuliah online temennya sikit,-6,negative
2,tumbenan td kuliah online dosennya minta join ...,-6,negative
3,nangis krn kecapean kuliah online,-9,negative
4,apa hanya aku yang merasa semenjak kuliah onli...,-2,negative


In [329]:
df.polarity.value_counts()

negative    12733
positive     8441
neutral      2524
Name: polarity, dtype: int64

In [330]:
max(df.polarity_score)

29

In [331]:
min(df.polarity_score)

-48

In [332]:
df_positive = df.loc[(df.polarity_score >= 5)]
df_positive = df_positive.sort_values(by=['polarity_score'], ascending=False)
df_positive = df_positive.head(2500)

print(len(df_positive))

2500


In [333]:
df_positive.head()

,text_clean,polarity_score,polarity
13210,good morning semangat yang kerja hari ini semo...,29,positive
12253,alhamdulillah seminggu ini padat kegiatan nya ...,29,positive
7785,good morning 🤗✨ thanks allah its friday 💕😇 yu...,28,positive
16350,selamat pagi dunia selamat pagi para mutual ku...,27,positive
2028,bersyukur masih bisa kuliah bersyukur masih da...,26,positive


In [334]:
df_negative = df.loc[(df.polarity_score <= -10)]
df_negative = df_negative.sort_values(by=['polarity_score'], ascending=True)
df_negative = df_negative.head(2500)

print(len(df_negative))

2500


In [335]:
df_negative = df_negative.sort_values(by=['polarity_score'], ascending=True)
df_negative.head()

,text_clean,polarity_score,polarity
9013,kuliah online apa yg kau dapat boros kuota mat...,-48,negative
1560,enjoy gimana nder kuliah online yang kudapatka...,-43,negative
5056,eh corona please udahan yuk cape kuliah online...,-43,negative
16230,pusing gw anjing gakuat gw kuliah online anjin...,-38,negative
3763,ambis boleh tapi pls lah jangan egois sampe to...,-38,negative


In [336]:
df_netral = df.loc[(df.polarity_score == 0)]
df_netral = df_netral.sample(2500)
len(df_netral)

2500

In [337]:
df_netral.head()

,text_clean,polarity_score,polarity
10517,gakuad kuliah online 🙃,0,neutral
11494,adek w kuliah online sambil olahraga dong,0,neutral
17221,bentar lagi kuliah online aargh diriku blm siap😠,0,neutral
20689,masih kuliah online kah,0,neutral
12727,kuliah online bukannya nambah pinter malah nam...,0,neutral


In [338]:
df_train = pd.concat([df_positive, df_negative, df_netral])

### **Example Review Dataset**

In [274]:
!gdown 1IuIe6AVjb5AGO6b4mvv0RaHcfYdCTNLT
!gdown 1AvOvYBr_jmZhspg4zjIltHahYlTRwHox

Downloading...
From: https://drive.google.com/uc?id=1IuIe6AVjb5AGO6b4mvv0RaHcfYdCTNLT
To: /content/telkomuniversity.xlsx
100% 69.1k/69.1k [00:00<00:00, 1.89MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AvOvYBr_jmZhspg4zjIltHahYlTRwHox
To: /content/telu.xlsx
100% 197k/197k [00:00<00:00, 2.09MB/s]


In [275]:
data_1 = pd.read_excel('/content/telkomuniversity.xlsx')
data_1 = data_1[['Text']]

data_2 = pd.read_excel('/content/telu.xlsx')
data_2 = data_2[['Text']]

In [276]:
df_all = pd.concat([data_1, data_2])
df_all = df_all.drop_duplicates()

print("Total Data :", len(df_all))
df_all.head()

Total Data : 92


,Text
0,[LOWONGAN KERJA]\nList perusahaan yang sedang ...
1,[ FULL TIME ]\nFakultas Ekonomi dan Bisnis\n\n...
2,RT @IndonesiaDili : Universitas Telkom akan se...
3,"Halo nder, ada yg ikut beasiswa telkom univers..."
4,yang daftar telyu mau tanya dong. kan di web t...


## **Text Preprocessing**

In [277]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

In [278]:
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p2")

In [279]:
kata_baku = pd.read_csv('https://raw.githubusercontent.com/ShinyQ/One-Click-Sentiment_BE/main/app/dataset/Kamu-Alay.csv')
kata_baku = kata_baku.set_index("kataAlay")["kataBaik"].to_dict()

stop_words = list(pd.read_csv('https://raw.githubusercontent.com/ShinyQ/E-Wallet-Sentiment-Analysis_IFest-Unpad-2021/main/dataset/processed/stopwords_id_satya.txt', header=None)[0])

In [339]:
def cleansing(text):
    text = str(text)
    print("Text awal :\n\n", text)
    
    # Mengubah setiap kata menjadi lowercase
    text =  text.lower() 

    # Menghapus Link Dengan Pattern http/https dan www
    text = re.sub(r'http\S+', '', text)
    text = re.sub('(@\w+|#\w+)', '', text)

    # Menghapus Tag HTML
    text = re.sub('<.*?>', '', text)

    # Menghapus Tanda Baca Seperti Titik Dan Koma
    # text = text.translate(str.maketrans(' ', ' ', string.punctuation))

    # Menghapus Karakter Selain Huruf a-z dan A-Z
    text = re.sub('[^a-zA-Z]', ' ', text)

    # Mengganti baris baru (enter) dengan spasi
    text = re.sub("\n", " ", text)

    # Menghapus Spasi Yang Lebih Dari Satu
    text = re.sub('(s{2,})', ' ', text)
    
    print("\nHasil Prerocessing :", text)

    # Menghapus kata yang mengandung judul topik dan kata yang terdapat pada stopwords indonesia
    temp_text_split = []
    final_text = []
    text_split = text.split(' ')

    for i in range(len(text_split)):
        if text_split[i] not in stop_words and text_split[i] != 'ptn':
              if text_split[i] in kata_baku and text_split[i] != 'telkom':
                  text_split[i] = kata_baku[text_split[i]]
              
              if len(list(str(text_split[i]))) > 3:   
                 temp_text_split.append(str(text_split[i]))
    
    for i in range(len(temp_text_split)):
        if temp_text_split not in final_text:
            final_text.append(str(temp_text_split[i]))

    text = ' '.join(final_text)

    text = stopword.remove(text)
    print("\nHasil Penghapusan Stopword :", text)

    text = stemmer.stem(text)
    print("\nHasil Stemming :", text)

    # Mengembalikan Hasil Preprocessing Text
    return text

In [315]:
def tokenisasi(text):
    tokenize = tokenizer(text)

    print("\nHasil Encode:", tokenizer.encode(text))
    print("\nTokenisasi IndoBERT:", tokenizer.convert_ids_to_tokens(tokenizer.encode(text)))

### Text Preprocessing Result Example

In [316]:
text = cleansing(df_all['Text'].values[22])
tokenisasi(text)

Text awal :

 [H-34]

• review materi fisika inti
• trigonometri
• ngelanjutin to mipi

hari ini aku juga ada tes telkom university. wish me luck!!🤩 https://t.co/pfAAYbaZDk

Hasil Prerocessing :  h        review materi fisika inti   trigonometri   ngelanjutin to mipi  hari ini aku juga ada tes telkom university  wish me luck    

Hasil Penghapusan Stopword : review materi fisika inti trigonometri ngelanjutin tryout mipi hari telkom university wish luck

Hasil Stemming : review materi fisika inti trigonometri ngelanjutin tryout mipi hari telkom university wish luck

Hasil Encode: [2, 4372, 1616, 5940, 4865, 22363, 994, 15988, 10293, 700, 17, 13821, 2187, 3277, 6161, 406, 4808, 6537, 28390, 27903, 3]

Tokenisasi IndoBERT: ['[CLS]', 'review', 'materi', 'fisika', 'inti', 'trig', '##onom', '##etri', 'ngel', '##anjut', '##in', 'try', '##out', 'mi', '##pi', 'hari', 'telkom', 'university', 'wish', 'luck', '[SEP]']


In [317]:
text = cleansing(df_all['Text'].values[32])
tokenisasi(text)

Text awal :

 btw gais, besok aku tes untuk masuk telkom university, mohon doanya yaa

Hasil Prerocessing : btw gais  besok aku tes untuk masuk telkom university  mohon doanya yaa

Hasil Penghapusan Stopword : ngomong-ngomong gais besok masuk telkom university mohon doanya

Hasil Stemming : ngomong-ngomong gais besok masuk telkom university mohon doa

Hasil Encode: [2, 8266, 30469, 8266, 525, 37, 5839, 804, 4808, 6537, 2903, 4157, 3]

Tokenisasi IndoBERT: ['[CLS]', 'ngomong', '-', 'ngomong', 'ga', '##is', 'besok', 'masuk', 'telkom', 'university', 'mohon', 'doa', '[SEP]']


In [318]:
text = cleansing(df_all['Text'].values[55])
tokenisasi(text)

Text awal :

 Gapyear! Menurut kalian bagus ga sih jurusan manajemen bisnis telekomunikasi dan informasi Telkom university?

Hasil Prerocessing : gapyear  menurut kalian bagus ga sih jurusan manajemen bisnis telekomunikasi dan informasi telkom university 

Hasil Penghapusan Stopword : gapyear bagus jurusan manajemen bisnis telekomunikasi informasi telkom university

Hasil Stemming : gapyear bagus jurus manajemen bisnis telekomunikasi informasi telkom university

Hasil Encode: [2, 11231, 6885, 12, 1305, 10181, 2110, 996, 9361, 683, 4808, 6537, 3]

Tokenisasi IndoBERT: ['[CLS]', 'gap', '##ye', '##ar', 'bagus', 'jurus', 'manajemen', 'bisnis', 'telekomunikasi', 'informasi', 'telkom', 'university', '[SEP]']


In [ ]:
df_all['text_clean'] = df_all['Text'].apply(cleansing)
df_all.replace("", float("NaN"), inplace=True)
df_all.dropna(inplace=True)
df_all.to_csv('crawl_result.csv', index=False)

In [320]:
df_all['text_clean'].sample(5).values

array(['terima kasih banyak ucap atas dukung partisipasi jadi sponsor acara kampus ramadhan selenggara april fast harap bekerjasama event ikut terima kasih',
       'lebih baik informatika telkom university computer science bus bandung',
       'telkom university maba sudah grup kabarin',
       'offense terima telkom university jalur beasiswa beasiswa unggul prodi mbti actually sucks',
       'universitas telkom selenggara sesi informasi terima mahasiswa baru beasiswa timor leste onlen zoom hari rabu pukul silah daftar link'],
      dtype=object)

In [ ]:
df_train['text_clean'] = df_train['text_clean'].apply(cleansing)
df_train.replace("", float("NaN"), inplace=True)
df_train.dropna(inplace=True)

df_train = df_train[['text_clean', 'polarity']]
df_train.to_csv('df_train.csv', index=False)

In [341]:
len(df_train)

7500